<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-2/Session_2_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.nn import relu,softmax
from tensorflow.keras.optimizers import SGD




In [0]:
# https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
# https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip





In [2]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip \
    -O /tmp/smallData.zip

--2020-03-26 18:47:22--  https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/smallData.zip [following]
--2020-03-26 18:47:23--  https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/smallData.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7426985 (7.1M) [application/zip]
Saving to: ‘/tmp/smallData.zip’

/tmp/smallData.zip  100%[===================>]   7.08M  --.-KB/s    in 0.1s    

2020-03-26 18:47:26 (51.5 MB/s) - ‘/tmp/smallData.zip’ saved [7426985/7426985]



In [0]:
import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/smallData.zip","/tmp")

In [4]:
if os.path.exists('/tmp/smallData'):
    print(os.listdir('/tmp/smallData'))


['train', '.gitignore', 'test']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)
CLASSES_COUNT = 2;
epochs = 50

In [7]:
#define the sub folders for both training and test
training = os.path.join("/tmp/smallData",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           )

Found 95 images belonging to 2 classes.


In [14]:
model = Sequential()
model.add(Flatten(input_shape=targetSize_withdepth))
model.add(Dense(1024,activation=relu))
model.add(Dense(512,activation=relu))
model.add(Dense(CLASSES_COUNT,activation=softmax))
model.compile(optimizer=SGD(),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              30721024  
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 1026      
Total params: 31,246,850
Trainable params: 31,246,850
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit_generator(generator=train_generator,epochs=epochs)


In [0]:
from sklearn.metrics import confusion_matrix, classification_report
def test(generator, model):
  predictions = model.predict_generator(generator)
  row_index = predictions.argmax(axis=1)
  target_names = generator.class_indices.keys()
  print(target_names)
  print(confusion_matrix(generator.classes, row_index))
  print('Classification Report')
  print(classification_report(generator.classes, row_index, target_names=target_names))

In [19]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/train",
                                                         target_size=(100,100),
                                                         shuffle=False)
test(generator=test_generator, model=model)

Found 95 images belonging to 2 classes.
ERROR! Session/line number was not unique in database. History logging moved to new session 59


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'pie_chart'])
[[39  0]
 [ 0 56]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       1.00      1.00      1.00        39
   pie_chart       1.00      1.00      1.00        56

    accuracy                           1.00        95
   macro avg       1.00      1.00      1.00        95
weighted avg       1.00      1.00      1.00        95



In [20]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/test",
                                                         target_size=(100,100),
                                                         shuffle=False)

Found 37 images belonging to 2 classes.


In [21]:
test(generator=test_generator, model=model)

dict_keys(['bar_chart', 'pie_chart'])
[[15  4]
 [ 4 14]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       0.79      0.79      0.79        19
   pie_chart       0.78      0.78      0.78        18

    accuracy                           0.78        37
   macro avg       0.78      0.78      0.78        37
weighted avg       0.78      0.78      0.78        37



/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
